In [2]:
import pandas as pd

In [ ]:
data = pd.read_excel("DataM(2).xlsx")
# Create a DataFrame
df = pd.DataFrame(data)

# Unpivot the table using pd.melt
df_unpivoted = pd.melt(
    df,
    id_vars=["Idicators"],  # Keep the Indicators column as is
    var_name="Year",  # Name of the new column for years
    value_name="Value"  # Name of the new column for values
)

# Display the unpivoted DataFrame
df_unpivoted

,Indicators,Year,Value
0,Revenue,Companies,NVDA US
1,Net Income/Net Profit (Losses),Companies,NVDA US
2,Total Assets,Companies,NVDA US
3,Current Market Cap,Companies,NVDA US
4,Volume,Companies,NVDA US
...,...,...,...
12252,Total Liabilities,2023,261.2497
12253,Total Debt,2023,61.6868
12254,Quick ratio,2023,4.1351
12255,Cash and equiv,2023,123.812
